In [1]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import lightning.pytorch as pl
from lightning.pytorch.callbacks import RichProgressBar, Timer

# Add the prime_torch file to the system path so we can import it
import sys
sys.path.append("/glade/u/home/cobrien/prime/prime_lib/primesw")
from data import SWDataset, SWDataModule
from prime_torch import crps, SWRegressor

DATAPATH = '~/data/'

In [2]:
# test_dataframe = pd.DataFrame([])
# test_dataframe['time'] = pd.date_range(
#     pd.to_datetime('20150902 00:00:00+0000'),
#     pd.to_datetime('20250101 00:00:00+0000'),
#     freq = '100s'
# )
# test_dataframe['a'] = np.arange(len(test_dataframe)) # Fake input
# test_dataframe['b'] = test_dataframe['a'] * 2 # Fake target
# test_dataframe['x_pos'] = np.arange(len(test_dataframe)) * 0.1
# test_dataframe['y_pos'] = np.arange(len(test_dataframe)) * 0.2
# test_dataframe['z_pos'] = np.arange(len(test_dataframe)) * 0.3
# test_dataframe.to_hdf("~/data/prime/test.h5", key = 'lineartest')

data = pd.read_hdf(DATAPATH + 'combined_data.h5', key = '1min_mms_wind')
# list(data.columns)

In [4]:
# data.loc[(data['modified_named_label'] == 'solar wind')&(data['stable'] == 1), :].iloc[int(0.8*247215):, :]
data['Epoch'].diff().unique()

<TimedeltaArray>
[              NaT, '0 days 00:01:00', '0 days 02:28:00', '0 days 14:06:00',
 '0 days 13:47:00', '0 days 13:28:00', '1 days 13:04:00', '0 days 11:15:00',
 '0 days 10:58:00', '0 days 10:41:00',
 ...
 '2 days 00:32:00', '2 days 01:56:00', '2 days 15:12:00', '2 days 15:17:00',
 '2 days 19:48:00', '3 days 00:58:00', '3 days 02:14:00', '3 days 04:31:00',
 '3 days 07:12:00', '3 days 09:19:00']
Length: 750, dtype: timedelta64[ns]

In [5]:
pd.date_range(data['Epoch'].min(), data['Epoch'].max(), freq = "1min")

DatetimeIndex(['2015-09-01 09:45:00+00:00', '2015-09-01 09:46:00+00:00',
               '2015-09-01 09:47:00+00:00', '2015-09-01 09:48:00+00:00',
               '2015-09-01 09:49:00+00:00', '2015-09-01 09:50:00+00:00',
               '2015-09-01 09:51:00+00:00', '2015-09-01 09:52:00+00:00',
               '2015-09-01 09:53:00+00:00', '2015-09-01 09:54:00+00:00',
               ...
               '2023-06-23 06:36:00+00:00', '2023-06-23 06:37:00+00:00',
               '2023-06-23 06:38:00+00:00', '2023-06-23 06:39:00+00:00',
               '2023-06-23 06:40:00+00:00', '2023-06-23 06:41:00+00:00',
               '2023-06-23 06:42:00+00:00', '2023-06-23 06:43:00+00:00',
               '2023-06-23 06:44:00+00:00', '2023-06-23 06:45:00+00:00'],
              dtype='datetime64[ns, UTC]', length=4106701, freq='min')

In [10]:
list(data.columns)

['Epoch',
 'probe',
 'ratio_max_width',
 'ratio_high_low',
 'norm_Btot',
 'small_energy_mean',
 'large_energy_mean',
 'temp_total',
 'r_gse_x',
 'r_gse_y',
 'r_gse_z',
 'r_gsm_x',
 'r_gsm_y',
 'r_gsm_z',
 'mlat',
 'mlt',
 'raw_named_label',
 'modified_named_label',
 'transition_name',
 'mms1_dis_bulkv_gse_fast_0',
 'mms1_dis_bulkv_gse_fast_1',
 'mms1_dis_bulkv_gse_fast_2',
 'mms1_dis_numberdensity_fast',
 'SW_table',
 'mms1_dis_energyspectr_omni_fast_0',
 'mms1_dis_energyspectr_omni_fast_1',
 'mms1_dis_energyspectr_omni_fast_2',
 'mms1_dis_energyspectr_omni_fast_3',
 'mms1_dis_energyspectr_omni_fast_4',
 'mms1_dis_energyspectr_omni_fast_5',
 'mms1_dis_energyspectr_omni_fast_6',
 'mms1_dis_energyspectr_omni_fast_7',
 'mms1_dis_energyspectr_omni_fast_8',
 'mms1_dis_energyspectr_omni_fast_9',
 'mms1_dis_energyspectr_omni_fast_10',
 'mms1_dis_energyspectr_omni_fast_11',
 'mms1_dis_energyspectr_omni_fast_12',
 'mms1_dis_energyspectr_omni_fast_13',
 'mms1_dis_energyspectr_omni_fast_14',
 'mm

In [ ]:
target_features = [
    'mms1_dis_bulkv_gse_fast_0', # V GSE X
    'mms1_dis_bulkv_gse_fast_1', # V GSE Y
    'mms1_dis_bulkv_gse_fast_2', # V GSE Z
    # 'mms1_dis_numberdensity_fast', # Ni
    # 'mms1_dis_temppara_fast', # Ti parallel to B
    # 'mms1_dis_tempperp_fast', # Ti perpendicular to B
    'mms1_des_numberdensity_fast', # Ne
    'mms1_fgm_b_gse_srvy_l2_0', # B GSE X
    'mms1_fgm_b_gse_srvy_l2_1', # B GSE Y
    'mms1_fgm_b_gse_srvy_l2_2', # B GSE Z
]
input_features = [
    'Np', # Ni
    'V_GSE_0', # V GSE X
    'V_GSE_1', # V GSE Y
    'V_GSE_2', # V GSE Z
    'THERMAL_SPD', # Vth
    'BGSE_0', # B GSE X
    'BGSE_1', # B GSE Y
    'BGSE_2', # B GSE Z
    'PGSE_0', # Wind Position GSE X
    'PGSE_1', # Wind Position GSE Y
    'PGSE_2', # Wind Position GSE Z
]
position_features = [
    'mms1_mec_r_gse_0', # MMS Position GSE X
    'mms1_mec_r_gse_1', # MMS Position GSE Y
    'mms1_mec_r_gse_2', # MMS Position GSE Z
]

trn_bounds = ['20151007 12:08:00+0000', '20211217 04:33:00+0000']
tst_bounds = ['20211217 04:34:00+0000', '20221206 11:21:00+0000']
val_bounds = ['20221206 11:22:00+0000', '20230601 14:21:00+0000']
datamodule = SWDataModule(
    target_features = target_features,
    input_features = input_features,
    position_features = position_features,
    region = 'solar wind',
    cadence = '100s',
    interpolate = True,
    window = 100,
    stride = 10,
    interp_frac = 0.1,
    trn_bounds = trn_bounds,
    val_bounds = val_bounds,
    tst_bounds = tst_bounds,
    datastore = "~/data/combined_data.h5",
    key = "1min_mms_wind",
)
datamodule.setup()

In [ ]:
model = SWRegressor(
    in_dim = len(input_features),
    tar_dim = len(target_features),
    pos_dim = len(position_features),
    decoder_type = 'linear',
    encoder_type = 'rnn',
    lr_scheduler = 'cosine',
    decoder_hidden_layers = [4],
    encoder_hidden_dim = 4,
    pos_encoding_size=4,
    encoder_num_layers=1,
    loss='mae'
)

In [6]:
logger = pl.loggers.TensorBoardLogger(
    save_dir = "~/data/prime/tensorboard_logs",
    name = 'test_model',
)

In [7]:
trainer = pl.Trainer(
    accelerator='cpu',
    max_epochs=1,
    callbacks = [
        Timer(), 
        RichProgressBar(),
        # ModelCheckpoint(),
        ],
    logger = logger,
    # precision='16-true', #Lower the precision to not blow up memory
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/glade/work/cobrien/conda-envs/pt212gpu_conda/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [8]:
trainer.fit(model=model, datamodule=datamodule)

2025-09-25 09:15:48.455 | INFO     | data:setup:241 - Train dataloader is ready. Dataset size: 148220
2025-09-25 09:15:56.163 | INFO     | data:setup:259 - Validation dataloader is ready. Dataset size: 49334
2025-09-25 09:16:03.538 | INFO     | data:setup:277 - Test dataloader is ready. Dataset size: 49334


┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type              ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ trn_mae │ MeanAbsoluteError │      0 │ train │
│ 1 │ val_mae │ MeanAbsoluteError │      0 │ train │
│ 2 │ tst_mae │ MeanAbsoluteError │      0 │ train │
│ 3 │ encoder │ RecurrentEncoder  │     68 │ train │
│ 4 │ decoder │ LinearDecoder     │    175 │ train │
└───┴─────────┴───────────────────┴────────┴───────┘

Trainable params: 243                                                                                              
Non-trainable params: 0                                                                                            
Total params: 243                                                                                                  
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 11                                                                                          
Modules in eval mode: 0

Output()

/glade/work/cobrien/conda-envs/pt212gpu_conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/dat
a_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing 
the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.

ValueError: `self.log(val_mae, tensor([nan, nan, nan, nan, nan, nan, nan]))` was called, but the tensor must have a single element. You can try doing `self.log(val_mae, tensor([nan, nan, nan, nan, nan, nan, nan]).mean())`

In [ ]:
in_test = torch.rand((50,100,14))
tar_test = torch.rand((50,1))
pos_test = torch.rand((50,3))
out_test = model.forward(in_test, pos_test)
model.loss_fn(out_test, tar_test)

In [7]:
data

,Epoch,probe,ratio_max_width,ratio_high_low,norm_Btot,small_energy_mean,large_energy_mean,temp_total,r_gse_x,r_gse_y,...,BGSE_0,BGSE_1,BGSE_2,PGSM_0,PGSM_1,PGSM_2,PGSE_0,PGSE_1,PGSE_2,stable
0,2015-09-01 09:45:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5657.4790,27957.251664,48908.731178,...,2.525612,-0.741759,-4.199306,242.518272,75.762352,-39.210161,242.518272,85.105907,5.861351,1.0
1,2015-09-01 09:46:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5176.8270,27939.570430,49036.430375,...,2.715868,-0.589167,-4.121939,242.519119,75.752541,-39.226168,242.519119,85.104527,5.861685,1.0
2,2015-09-01 09:47:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5627.4062,27921.706142,49163.804439,...,2.883286,-0.662489,-4.005026,242.519966,75.742836,-39.241957,242.519966,85.103146,5.862018,1.0
3,2015-09-01 09:48:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5544.2476,27903.587167,49290.724218,...,2.871315,-0.728962,-3.972454,242.520805,75.733242,-39.257523,242.520805,85.101765,5.862352,1.0
4,2015-09-01 09:49:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5501.0570,27885.287357,49417.320561,...,3.085248,-0.934587,-3.687918,242.521645,75.723759,-39.272861,242.521645,85.100380,5.862685,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909345,2023-06-23 06:41:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5691.1580,8383.837752,-55172.020080,...,2.842887,-3.107068,1.269845,254.961861,68.650379,4.595528,254.961861,67.931499,10.922728,1.0
909346,2023-06-23 06:42:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5835.5230,8438.776123,-55018.827470,...,3.334452,-2.769564,-0.016363,254.962456,68.651756,4.548562,254.962456,67.929707,10.922893,1.0
909347,2023-06-23 06:43:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5691.9937,8493.676110,-54865.365609,...,4.158662,-2.509459,-2.102036,254.963058,68.653095,4.501664,254.963058,67.927910,10.923058,1.0
909348,2023-06-23 06:44:00+00:00,mms1,0.0,0.0,0.0,0.0,0.0,5680.6720,8548.521321,-54711.525610,...,4.790656,-2.389047,-2.678434,254.963661,68.654400,4.454869,254.963661,67.926113,10.923223,1.0
